# Convolutional Neural Networks

### Importing Libraries

In [ ]:
import Pkg
Pkg.add("DataFrames")
Pkg.add("Flux")
Pkg.add("CSV")
Pkg.add("MLJ")
Pkg.add("CUDA")
Pkg.add("IterTools")
Pkg.add("ProgressMeter")
Pkg.add("Images")
Pkg.add("Augmentor")
Pkg.add("Glob")
Pkg.add("MLUtils")
Pkg.add("ImageShow")

    Updating registry at `/opt/julia/registries/General.toml`
   Resolving package versions...
   Installed Crayons ───────────────────── v4.1.1
   Installed DataAPI ───────────────────── v1.14.0
   Installed InlineStrings ─────────────── v1.4.0
   Installed PooledArrays ──────────────── v1.4.2
   Installed Tables ────────────────────── v1.10.0
   Installed TableTraits ───────────────── v1.0.1
   Installed IteratorInterfaceExtensions ─ v1.0.0
   Installed SentinelArrays ────────────── v1.3.18
   Installed Formatting ────────────────── v0.4.2
   Installed DataValueInterfaces ───────── v1.0.0
   Installed LaTeXStrings ──────────────── v1.3.0
   Installed InvertedIndices ───────────── v1.2.0
   Installed OrderedCollections ────────── v1.4.1
   Installed Reexport ──────────────────── v1.2.2
   Installed PrettyTables ──────────────── v2.2.2
   Installed SortingAlgorithms ─────────── v1.1.0
   Installed Missings ──────────────────── v1.1.0
   Installed StringManipulation ────────── v0.3.0
  

In [ ]:
using DataFrames
using CSV
using Flux
using MLJ
using CUDA
using IterTools: ncycle
using ProgressMeter
using Images
using Glob
using MLUtils
using Augmentor
using CUDA: CuIterator
using ImageShow

## Part 1 - Data Preprocessing

#### Check out our data

In [ ]:
dog_img_path = "data/catsndogs/training_set/dogs/dog.1992.jpg"

dog_img = Images.load(dog_img_path)

In [ ]:
size(dog_img)

In [ ]:
cat_img_path = "data/catsndogs/training_set/cats/cat.2701.jpg"

cat_img = Images.load(cat_img_path)

#### Look at them side by side

In [ ]:
mosaicview(dog_img, cat_img; nrow=1)

In [ ]:
typeof(dog_img)

In [ ]:
size(cat_img)

In [ ]:
IMAGE_HEIGHT = 64
IMAGE_WIDTH = 64
IMAGE_SIZE = (IMAGE_HEIGHT, IMAGE_WIDTH)
BATCH_SIZE = 32

#### Importing the Datasets

#### Training Set

In [ ]:
training_image_filenames = readdir(glob"*/*.jpg", "data/catsndogs/training_set/")

In [ ]:
#(X_train, X_test), (y_train, y_test) = partition((image_filenames, map(x -> String(rsplit(x, "/")[2])== "dog" ? 1.0 : 0.0, image_filenames)), 0.8, rng=43, multi=true)

In [ ]:
struct TrainingImageDataset
    files::Vector{String}
    labels::Vector{Float32}
end

In [ ]:
MLUtils.numobs(dataset::TrainingImageDataset) = length(dataset.files)

In [ ]:
train_image_transformation_pipeline = 
    Augmentor.ElasticDistortion(10,10,0.2,4,3,true) |> 
    #Augmentor.Zoom(0.8:0.1:1.2) |>
    Augmentor.FlipX(0.5) |>
    Augmentor.Resize(IMAGE_SIZE...) |>
    Augmentor.SplitChannels() |>
    Augmentor.PermuteDims((3, 2, 1)) |>
    Augmentor.ConvertEltype(Float32)
    #Resize(64, 64)

function load_training_image(image_path)
    image = Images.load(image_path)
    #image = Images.imresize(image, (64, 64))
    image = augment(image, train_image_transformation_pipeline)
    #image = permutedims(convert(Array{Float32, 3}, channelview(image)), (2, 3, 1))
    #print(size(image))
    return image
end

MLUtils.getobs(data::TrainingImageDataset, i::Int) = load_training_image(data.files[i])

In [ ]:
training_image_data = TrainingImageDataset(training_image_filenames, map(x -> String(rsplit(x, "/")[2])== "dog" ? 1.0 : 0.0, training_image_filenames)) |> gpu

In [ ]:
training_loader = CuIterator(Flux.DataLoader((data=training_image_data, label=training_image_data.labels), collate=true, batchsize = 32, shuffle = true))

#### Test Set

In [ ]:
test_image_filenames = readdir(glob"*/*.jpg", "data/catsndogs/test_set/")

In [ ]:
struct TestImageDataset
    files::Vector{String}
    labels::Vector{Float32}
end

In [ ]:
MLUtils.numobs(dataset::TestImageDataset) = length(dataset.files)

In [ ]:
test_image_transformation_pipeline = 
    Augmentor.Resize(IMAGE_SIZE...) |>
    Augmentor.SplitChannels() |>
    Augmentor.PermuteDims((3, 2, 1)) |>
    Augmentor.ConvertEltype(Float32)

function load_test_image(image_path)
    image = Images.load(image_path)
    #image = Images.imresize(image, (64, 64))
    image = augment(image, test_image_transformation_pipeline)
    #image = permutedims(convert(Array{Float32, 3}, channelview(image)), (2, 3, 1))
    return image
end

MLUtils.getobs(data::TestImageDataset, i::Int) = load_test_image(data.files[i])

In [ ]:
test_image_data = TestImageDataset(test_image_filenames, map(x -> String(rsplit(x, "/")[2])== "dog" ? 1.0 : 0.0, test_image_filenames)) |> gpu

In [ ]:
test_loader = CuIterator(Flux.DataLoader((data=test_image_data, label=test_image_data.labels), collate = false, batchsize = 32, shuffle = true))

## Part 2 - Building the CNN

#### Step 1: Convolution

In [ ]:
conv_layer1 = Flux.Conv((3, 3), 3 => 32, relu)

#### Step 2: Pooling

In [ ]:
max_pool_layer1 = Flux.MaxPool((2, 2), stride = 2)

### Second Layer

In [ ]:
conv_layer2 = Flux.Conv((3, 3), 32 => 32, relu)
max_pool_layer2 = Flux.MaxPool((2, 2), stride = 2)

#### Step 3: Flattening

In [ ]:
flat_layer = Flux.flatten

#### Step 4: Full Connection

In [ ]:
dense_layer = Flux.Dense(6272 => 128, relu)

#### Step 5: Output Layer

In [ ]:
output_layer  = Dense(128 => 1, sigmoid)

#### Step 6: Put it together

In [ ]:
model = Chain(
    conv_layer1, 
    max_pool_layer1,
    conv_layer2,
    max_pool_layer2,
    flat_layer, 
    dense_layer,
    output_layer
)

In [ ]:
#Place the model onto the gpu
model = fmap(cu, model)

#### Test the model

In [ ]:
count = 1
for batch in training_loader
    println(count)
    #println(length(batch))
    #println(size(batch.data))
    #println(typeof(batch))
    @show size(batch.data[:, :, :, 1] |> cpu)
    @show ImageShow.gif(batch.data[:, :, :, 1] |> cpu)
    count += 1
    output1 = model(batch.data)
    println(output1 .> 0.5, " ", size(output1))
    break
end

In [ ]:
optimizer = Flux.setup(Flux.Adam(0.01), model) |> gpu
loss(m, x) = Flux.logitbinarycrossentropy(m(x.data), transpose(x.label)) |> gpu

#### Train the model

In [ ]:
number_epochs = 10

@showprogress for epoch in 1:number_epochs
    Flux.train!(loss, model, training_loader, optimizer) 
end

## Part 4 - Single Predictions

In [ ]:
test_single_image_1_path = "data/catsndogs/single_prediction/cat_or_dog_1.jpg"
test_single_image_2_path = "data/catsndogs/single_prediction/cat_or_dog_2.jpg"

test_single_image_1 = load_test_image(test_single_image_1_path)
test_single_image_2 = load_test_image(test_single_image_2_path)
@show size(test_single_image_1)

img1 = Images.load(test_single_image_1_path)
img2 = Images.load(test_single_image_2_path)
mosaicview(img1, img2; nrow=1)

In [ ]:
print(size(reshape(test_single_image_1, (size(test_single_image_1)..., 1))))
single_predict_1 = model(reshape(test_single_image_1, (size(test_single_image_1)..., 1)) |> gpu)

In [ ]:
single_predict_2 = model(reshape(test_single_image_2, (size(test_single_image_2)..., 1)) |> gpu)